In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
cam = 1
fps = 60

model_version = "my_model_13.0.keras"
Im_Width = 128
Im_Height = 128

In [3]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [4]:
def measure_brightness(frame):
    cropped_frame = crop_image(frame, 120, 0, 400, 250)
    gray = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    brightness = np.mean(gray)  # Compute the mean intensity
    return brightness

In [5]:
def select_best_frame(frame_buffer):
    best_frame = None
    best_score = -1 

    for frame in frame_buffer:
        score = measure_brightness(frame)
        if score > best_score:
            best_score = score
            best_frame = frame

    return best_frame

In [6]:
def process_frame(frame, target_size):
    
    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [7]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

    # Make a prediction
    predictions = model.predict(frame_array)  # Outputs probabilities for each class
    predicted_class = np.argmax(predictions[0])  # Get the index of the class with the highest probability
    confidence = predictions[0][predicted_class]  # Confidence score for the predicted class

    # Map class indices to labels
    class_labels = {0: "Defective", 2: "Non-Defective", 1: "No Bottle"} 
    predicted_label = class_labels[predicted_class]

    return predicted_label, confidence

In [8]:
def capture(cam, fps, model_path, target_size=(Im_Width, Im_Height)):
    cap = cv2.VideoCapture(cam)  # Open webcam
    # Set desired frame rate
    desired_fps = fps 
    cap.set(cv2.CAP_PROP_FPS, desired_fps)

    frame_buffer = []
    max_frames = 30  # Number of frames to evaluate

    # Load the model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:
        
        frame = cap.read()

        frame_buffer.append(frame)

        # Process the buffer when it's full
        if len(frame_buffer) == max_frames:
            best_frame = select_best_frame(frame_buffer)
            frame_buffer = []  # Clear the buffer

            # Process and predict
            feedback, confidence = predict_frame(best_frame, model, target_size)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            cv2.putText(best_frame, f"Prediction: {feedback}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(best_frame, f"Confidence: {confidence:.2f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Display the best frame
            cv2.imshow("Best Frame", best_frame)

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [9]:
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    capture(cam, fps, model_path, target_size=(Im_Width, Im_Height))

Loading model...
Model loaded successfully.


TypeError: tuple indices must be integers or slices, not tuple